In [1]:
import torch

In [2]:
def corr2d(X, K):
    kh, kw = K.shape
    xh, xw = X.shape
    
    Y = torch.zeros((xh-kh+1), (xw-kw+1))
    
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+kh, j:j+kw] *K).sum()
    
    return Y

In [3]:
sum([2,3])

5

In [4]:
def corr2d_multi_in(X,K):
    return sum(corr2d(x,k) for x,k in zip(X,K))

In [5]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
print(X.shape, K.shape)
corr2d_multi_in(X, K)

torch.Size([2, 3, 3]) torch.Size([2, 2, 2])


tensor([[ 56.,  72.],
        [104., 120.]])

In [6]:
K = torch.normal(0,1,(1,1,1))
corr2d_multi_in(X,K).shape

torch.Size([3, 3])

In [7]:
[corr2d(x,k) for x,k in zip(X,K)]

[tensor([[ 0.0000, -0.1945, -0.3890],
         [-0.5835, -0.7780, -0.9725],
         [-1.1670, -1.3616, -1.5561]])]

In [8]:
def corr2d_multi_in_out(X,K):
    return torch.stack([corr2d_multi_in(X,k) for k in K],0)

In [9]:
K.shape

torch.Size([1, 1, 1])

In [10]:
K = torch.stack((K, K+1, K+2),0)
K.shape

torch.Size([3, 1, 1, 1])

In [11]:
corr2d_multi_in_out(X,K)

tensor([[[ 0.0000, -0.1945, -0.3890],
         [-0.5835, -0.7780, -0.9725],
         [-1.1670, -1.3616, -1.5561]],

        [[ 0.0000,  0.8055,  1.6110],
         [ 2.4165,  3.2220,  4.0275],
         [ 4.8330,  5.6384,  6.4439]],

        [[ 0.0000,  1.8055,  3.6110],
         [ 5.4165,  7.2220,  9.0275],
         [10.8330, 12.6384, 14.4439]]])

In [12]:
def corr2d_multi_in_out_1x1(X,K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K,X)
    return Y.reshape((c_o, h, w))

In [13]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

In [14]:
Y1 = corr2d_multi_in_out_1x1(X,K)
Y2 = corr2d_multi_in_out(X,K)
print(Y1, Y2)

print(Y1==Y2)

tensor([[[-0.2191,  5.6725, -2.0104],
         [ 1.6895, -1.0932, -2.2256],
         [-1.5367, -3.5512,  2.4376]],

        [[-2.0566,  4.4244, -1.6383],
         [ 0.4864, -0.1387, -0.1849],
         [ 0.9553, -3.5452, -0.6145]]]) tensor([[[-0.2191,  5.6725, -2.0104],
         [ 1.6895, -1.0932, -2.2256],
         [-1.5367, -3.5512,  2.4376]],

        [[-2.0566,  4.4244, -1.6383],
         [ 0.4864, -0.1387, -0.1849],
         [ 0.9553, -3.5452, -0.6145]]])
tensor([[[True, True, True],
         [True, True, True],
         [True, True, True]],

        [[True, True, True],
         [True, True, True],
         [True, True, True]]])


### Exercises

1. Assume that we have two convolution kernels of size $k_1$ and $k_2$, respectively (with no nonlinearity in between).
    1. Prove that the result of the operation can be expressed by a single convolution.
    * not sure
    1. What is the dimensionality of the equivalent single convolution?
    * dont know
    1. Is the converse true?
    * dont know
    
1. Assume an input of shape $c_i\times h\times w$ and a convolution kernel of shape $c_o\times c_i\times k_h\times k_w$, padding of $(p_h, p_w)$, and stride of $(s_h, s_w)$.
    1. What is the computational cost (multiplications and additions) for the forward propagation?
    1. What is the memory footprint?
    
    1. What is the memory footprint for the backward computation?
    * dont know
    
    1. What is the computational cost for the backpropagation?
    * dont know
    
1. By what factor does the number of calculations increase if we double the number of input channels $c_i$ and the number of output channels $c_o$? What happens if we double the padding?

    * calculations would be multiplied by $c_i$  and if $c_o$ it would be multiplied by length of $c_o$
    * if we double the padding then calculation would be h-kh+ph* 2,w - kw + pw * 2
    
1. If the height and width of a convolution kernel is $k_h=k_w=1$, what is the computational complexity of the forward propagation?

    * h - kh + 1, w - kw + 1 =  h, w
    
1. Are the variables `Y1` and `Y2` in the last example of this section exactly the same? Why?
    
    * it cameout same for me !
    
1. How would you implement convolutions using matrix multiplication when the convolution window is not $1\times 1$?
    
    * DONT KNOW. 


In [15]:
# 1

k1 = torch.normal(0,1,(2,3,2,2))
k2 = torch.normal(0,1,(2,3,2,2))

X = torch.normal(0,1,(3,3,3))

In [16]:
Y = corr2d_multi_in_out(X,k1)
print(Y)

corr2d_multi_in_out(Y, k2)

tensor([[[-4.4373,  0.7733],
         [ 2.7726,  0.0287]],

        [[-1.4538, -2.1907],
         [-3.5141, -6.7401]]])


tensor([[[17.7520]],

        [[-4.3148]]])

In [17]:
k1 = k1 + k2

In [18]:
corr2d_multi_in_out(X, k1)

tensor([[[  1.8279, -11.1181],
         [  4.4133,   2.9503]],

        [[ -1.7347,  -2.0985],
         [ -4.2719, -13.0135]]])

In [19]:
#6

def corr2d_multi_in_out_nxn(X,K):
    c_i , h, w = X.shape
    c_o, _, kh, kw = K.shape
    
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o * kh * kw, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, -1))

In [20]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 2, 2))

In [21]:
Y2 = corr2d_multi_in_out(X, K)
Y2

tensor([[[ 2.2263,  3.6681],
         [-1.1259, -1.3209]],

        [[ 7.2328,  3.0686],
         [-3.7975,  1.4645]]])

In [22]:
try:
    Y1 = corr2d_multi_in_out_nxn(X,K)
    print(Y1)
except Exception as e:
    print(e)

tensor([[-2.1815, -0.5035, -0.4300,  1.6996,  0.6384,  2.6970,  0.5897,  0.1399,
          3.0734, -0.6737, -1.6015, -0.7622,  2.7148, -0.2910,  2.4581, -1.3849,
         -1.0349, -0.5919, -3.2290, -2.5474, -0.3056,  2.1638,  1.2308,  3.7177,
          1.4757, -1.7264,  2.9952,  0.7533,  3.9835, -0.2855, -0.5765, -0.6120,
         -0.8988, -0.8695,  3.8986,  2.3110],
        [ 0.8602,  0.1394,  0.9779, -2.8848,  0.3971, -2.7169,  1.6146, -0.5404,
         -1.0008,  1.3272,  0.3577, -0.1431,  0.0679, -0.7134, -0.8180, -1.2816,
          0.1790, -1.9554,  0.7060, -3.7375,  0.6292, -0.6867,  0.2309, -1.0002,
          0.5955, -4.1104, -4.4322, -1.7866, -0.0259, -0.2931,  1.1082,  0.5658,
          1.9996,  0.6516,  0.4604,  2.8930]])


### Max poolinglayer

In [26]:
def find_mean(Y_sub):
    total = 0
    counter = 0
    for i in range(Y_sub.shape[0]):
        for j in range(Y_sub.shape[1]):
            total +=Y_sub[i][j]
            counter += 1
    return total/counter

In [27]:
find_mean(torch.normal(0,1,(2,2)))

tensor(-0.9042)

In [38]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i,j] = X[i:i+p_h, j:j+p_h].max()
            elif mode == 'avg':
                Y[i,j] = X[i:i+p_h, j:j+p_h].mean()
    return Y

In [39]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])

X1 = torch.normal(0,1,(3,3))

print(X1.shape, X.shape)

torch.Size([3, 3]) torch.Size([3, 3])


In [33]:
X, X1

(tensor([[0., 1., 2.],
         [3., 4., 5.],
         [6., 7., 8.]]),
 tensor([[ 2.0692,  2.6693, -0.2633],
         [-0.0327,  0.1380,  2.0699],
         [ 0.5216,  0.3013, -0.5565]]))

In [34]:
X2 = torch.normal(0,1, (1,9))
X2

tensor([[ 1.4278, -1.1468,  0.3987,  0.5388,  0.5868,  1.0652,  2.2685, -1.6087,
         -0.0755]])

In [40]:
pool2d(X, (2,2))

tensor([[4., 5.],
        [7., 8.]])

In [41]:
pool2d(X,(2,2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

### padding and stride in pooling

In [54]:
X = torch.arange(16, dtype=torch.float32).reshape(1,1,4,4)
X,X.shape

(tensor([[[[ 0.,  1.,  2.,  3.],
           [ 4.,  5.,  6.,  7.],
           [ 8.,  9., 10., 11.],
           [12., 13., 14., 15.]]]]),
 torch.Size([1, 1, 4, 4]))

In [44]:
from torch import nn
pool2d= nn.MaxPool2d(3)

In [45]:
pool2d(X)

z:\installs\anconda\envs\myenv\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


tensor([[[[10.]]]])

In [46]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [47]:
pool2d = nn.MaxPool2d((2,3),stride=(2,3), padding=(0,1))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

### Multiple channels

In [48]:
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [49]:
X+1

tensor([[[[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [55]:
X = torch.cat((X, X+1), 1)
X, X.shape

(tensor([[[[ 0.,  1.,  2.,  3.],
           [ 4.,  5.,  6.,  7.],
           [ 8.,  9., 10., 11.],
           [12., 13., 14., 15.]],
 
          [[ 1.,  2.,  3.,  4.],
           [ 5.,  6.,  7.,  8.],
           [ 9., 10., 11., 12.],
           [13., 14., 15., 16.]]]]),
 torch.Size([1, 2, 4, 4]))

In [56]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

### Exercises

1. Can you implement average pooling as a special case of a convolution layer? If so, do it.
2. Can you implement maximum pooling as a special case of a convolution layer? If so, do it.
3. What is the computational cost of the pooling layer? Assume that the input to the pooling
layer is of size c×h×w, the pooling window has a shape of ph ×pw with a padding of (ph, pw)
and a stride of (sh, sw).



4. Why do you expect maximum pooling and average pooling to work differently?

* because max pooling will give maximum value from the neighbors while average pooling would consider all the neighbors. I expect max pooling to be faster.

5. Do we need a separate minimum pooling layer? Can you replace it with another operation?

* This seems like a trick question, but one way is to multiply X by -1 and then do max pooling.

6. Is there another operation between average and maximum pooling that you could consider
(hint: recall the softmax)? Why might it not be so popular?

* taking average of the log and then computing the maximum value devide by sum of log. It might be computationally intensive.

In [66]:
# 1 # 2


K = torch.tensor([[0.25,0.25], [0.25,0.25]])

X = torch.tensor([[1,2],[3,4]])

print(X * K)

# what I am looking for is a kernel when multiplied with X gives me 4 as output
# one wayot achieve it would be

K = torch.tensor([[1.0,1.0],[1.0,1.0]])

print((X * K).max())
print((X *K).mean())

tensor([[0.2500, 0.5000],
        [0.7500, 1.0000]])
tensor(4.)
tensor(2.5000)


In [84]:
def pool_using_corr(X,K=K,mode="max"):
    p_h, p_w = K.shape
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if(mode=="max"):
                Y[i,j] = (X[i:i+p_h, j:j+p_w] * K).max()
            elif(mode=="avg"):
                Y[i,j] = (X[i:i+p_h, j:j:p_w] * K).mean()
    return Y

In [85]:
X = torch.arange(4, dtype=torch.float32).reshape(1,1,2,2) 
X1 = torch.arange(4, dtype=torch.float32).reshape(2,2)
X1

tensor([[0., 1.],
        [2., 3.]])

In [86]:
pool2d = nn.MaxPool2d(2)

pool2d(X)

tensor([[[[3.]]]])

In [87]:
pool_using_corr(X1)

tensor([[3.]])